In [92]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [93]:
data_dir = "./LakeModeling/1D_HeatMixing_PGDL/output/"
depth_steps = 25

print(os.getcwd())

C:\Users\ladwi\Documents\Projects\R\LakeModeling\1D_HeatMixing_PGDL\MCL


In [94]:
meterological_data_df = pd.read_csv("./../output/meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0
...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0


In [95]:
input_temp_df = pd.read_csv("./../output/temp_total04.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total04':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total04,depth
0,2009-06-04 01:00:00,11.641386,11.570472,1
1,2009-06-04 01:00:00,11.650003,11.570472,2
2,2009-06-04 01:00:00,11.650003,11.575860,3
3,2009-06-04 01:00:00,11.394495,11.393058,4
4,2009-06-04 01:00:00,11.123800,11.130929,5
...,...,...,...,...
106170,2011-08-01 23:00:00,7.038822,7.039837,21
106171,2011-08-01 23:00:00,6.215392,6.216154,22
106172,2011-08-01 23:00:00,5.400819,5.401328,23
106173,2011-08-01 23:00:00,4.590435,4.590689,24


In [96]:
depth_list.shape

(106175,)

In [97]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689


In [98]:
buoyancy_data_df = pd.read_csv("./../output/buoyancy.csv")
#Removing Nans
buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2009-06-04 01:00:00,0.000000,1
1,2009-06-04 01:00:00,0.000000,2
2,2009-06-04 01:00:00,0.000271,3
3,2009-06-04 01:00:00,0.000278,4
4,2009-06-04 01:00:00,0.000185,5
...,...,...,...
106170,2011-08-01 23:00:00,0.000329,21
106171,2011-08-01 23:00:00,0.000227,22
106172,2011-08-01 23:00:00,0.000127,23
106173,2011-08-01 23:00:00,0.000026,24


In [99]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026


In [100]:
out_diffusivity_df = pd.read_csv("./../output/diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2009-06-04 01:00:00,0.000037,1
1,2009-06-04 01:00:00,0.000037,2
2,2009-06-04 01:00:00,0.000021,3
3,2009-06-04 01:00:00,0.000021,4
4,2009-06-04 01:00:00,0.000024,5
...,...,...,...
106170,2011-08-01 23:00:00,0.000019,21
106171,2011-08-01 23:00:00,0.000022,22
106172,2011-08-01 23:00:00,0.000029,23
106173,2011-08-01 23:00:00,0.000037,24


In [101]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000,0.000037
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000,0.000037
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271,0.000021
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278,0.000021
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329,0.000019
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227,0.000022
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127,0.000029
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026,0.000037


In [102]:
out_temp_df = pd.read_csv("./../output/temp_diff01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff01,depth
0,2009-06-04 01:00:00,11.467275,1
1,2009-06-04 01:00:00,11.627332,2
2,2009-06-04 01:00:00,11.631393,3
3,2009-06-04 01:00:00,11.393058,4
4,2009-06-04 01:00:00,11.130929,5
...,...,...,...
106170,2011-08-01 23:00:00,7.039837,21
106171,2011-08-01 23:00:00,6.216154,22
106172,2011-08-01 23:00:00,5.401328,23
106173,2011-08-01 23:00:00,4.590689,24


In [103]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_diff01
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000,0.000037,11.467275
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000,0.000037,11.627332
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271,0.000021,11.631393
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278,0.000021,11.393058
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185,0.000024,11.130929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329,0.000019,7.039837
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227,0.000022,6.216154
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127,0.000029,5.401328
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026,0.000037,4.590689


In [104]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [105]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [106]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_diff01,day_of_year,time_of_day
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000,0.000037,11.467275,155,1
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000,0.000037,11.627332,155,1
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271,0.000021,11.631393,155,1
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278,0.000021,11.393058,155,1
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185,0.000024,11.130929,155,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329,0.000019,7.039837,213,23
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227,0.000022,6.216154,213,23
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127,0.000029,5.401328,213,23
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026,0.000037,4.590689,213,23


In [107]:
temp_mix02_df = pd.read_csv("./../output/temp_mix02.csv")

temp_mix02_df = temp_mix02_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix02_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix02_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix02':flattened_temp, 'depth':depth_list}
temp_mix02_df = pd.DataFrame(data=data)

temp_mix02_df

,time,temp_mix02,depth
0,2009-06-04 01:00:00,11.545011,1
1,2009-06-04 01:00:00,11.545011,2
2,2009-06-04 01:00:00,11.631393,3
3,2009-06-04 01:00:00,11.393058,4
4,2009-06-04 01:00:00,11.130929,5
...,...,...,...
106170,2011-08-01 23:00:00,7.039837,21
106171,2011-08-01 23:00:00,6.216154,22
106172,2011-08-01 23:00:00,5.401328,23
106173,2011-08-01 23:00:00,4.590689,24


In [108]:
final_df = final_df.merge(temp_mix02_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_diff01,day_of_year,time_of_day,temp_mix02
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000,0.000037,11.467275,155,1,11.545011
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000,0.000037,11.627332,155,1,11.545011
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271,0.000021,11.631393,155,1,11.631393
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278,0.000021,11.393058,155,1,11.393058
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185,0.000024,11.130929,155,1,11.130929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329,0.000019,7.039837,213,23,7.039837
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227,0.000022,6.216154,213,23,6.216154
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127,0.000029,5.401328,213,23,5.401328
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026,0.000037,4.590689,213,23,4.590689


In [109]:
temp_conv03_df = pd.read_csv("./../output/temp_conv03.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv03':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv03,depth
0,2009-06-04 01:00:00,11.570472,1
1,2009-06-04 01:00:00,11.570472,2
2,2009-06-04 01:00:00,11.575860,3
3,2009-06-04 01:00:00,11.393058,4
4,2009-06-04 01:00:00,11.130929,5
...,...,...,...
106170,2011-08-01 23:00:00,7.039837,21
106171,2011-08-01 23:00:00,6.216154,22
106172,2011-08-01 23:00:00,5.401328,23
106173,2011-08-01 23:00:00,4.590689,24


In [110]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.641386,11.570472,0.000000,0.000037,11.467275,155,1,11.545011,11.570472
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.570472,0.000000,0.000037,11.627332,155,1,11.545011,11.570472
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.650003,11.575860,0.000271,0.000021,11.631393,155,1,11.631393,11.575860
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.394495,11.393058,0.000278,0.000021,11.393058,155,1,11.393058,11.393058
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,36000000.0,11.123800,11.130929,0.000185,0.000024,11.130929,155,1,11.130929,11.130929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,7.038822,7.039837,0.000329,0.000019,7.039837,213,23,7.039837,7.039837
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,6.215392,6.216154,0.000227,0.000022,6.216154,213,23,6.216154,6.216154
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,5.400819,5.401328,0.000127,0.000029,5.401328,213,23,5.401328,5.401328
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,36000000.0,4.590435,4.590689,0.000026,0.000037,4.590689,213,23,4.590689,4.590689


In [111]:
temp_obs_df = pd.read_csv("./../output/observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

(106175,)
(106175,)
(106175,)


In [112]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total04,buoyancy,diffusivity,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03,obs_temp
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.641386,11.570472,0.000000,0.000037,11.467275,155,1,11.545011,11.570472,16.409
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.650003,11.570472,0.000000,0.000037,11.627332,155,1,11.545011,11.570472,16.480
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.650003,11.575860,0.000271,0.000021,11.631393,155,1,11.631393,11.575860,16.130
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.394495,11.393058,0.000278,0.000021,11.393058,155,1,11.393058,11.393058,15.827
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.123800,11.130929,0.000185,0.000024,11.130929,155,1,11.130929,11.130929,16.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,7.038822,7.039837,0.000329,0.000019,7.039837,213,23,7.039837,7.039837,11.407
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,6.215392,6.216154,0.000227,0.000022,6.216154,213,23,6.216154,6.216154,11.407
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,5.400819,5.401328,0.000127,0.000029,5.401328,213,23,5.401328,5.401328,11.407
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,4.590435,4.590689,0.000026,0.000037,4.590689,213,23,4.590689,4.590689,11.407


In [113]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../output/observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)

time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(106175,)
(106175,)
(106175,)


,time,input_obs,depth
0,2009-06-04 01:00:00,16.350,1
1,2009-06-04 01:00:00,16.426,2
2,2009-06-04 01:00:00,16.088,3
3,2009-06-04 01:00:00,15.789,4
4,2009-06-04 01:00:00,16.240,5
...,...,...,...
106170,2011-08-01 23:00:00,11.407,21
106171,2011-08-01 23:00:00,11.407,22
106172,2011-08-01 23:00:00,11.407,23
106173,2011-08-01 23:00:00,11.407,24


In [114]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total04,buoyancy,diffusivity,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03,obs_temp,input_obs
0,1,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.570472,0.000000,0.000037,11.467275,155,1,11.545011,11.570472,16.409,16.350
1,2,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.570472,0.000000,0.000037,11.627332,155,1,11.545011,11.570472,16.480,16.426
2,3,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.575860,0.000271,0.000021,11.631393,155,1,11.631393,11.575860,16.130,16.088
3,4,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.393058,0.000278,0.000021,11.393058,155,1,11.393058,11.393058,15.827,15.789
4,5,2009-06-04 01:00:00,10.715021,678.292163,-152.775961,-4.194743,0.0,0.255324,1.085796,0.00229,...,11.130929,0.000185,0.000024,11.130929,155,1,11.130929,11.130929,16.270,16.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106170,21,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,7.039837,0.000329,0.000019,7.039837,213,23,7.039837,7.039837,11.407,11.407
106171,22,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,6.216154,0.000227,0.000022,6.216154,213,23,6.216154,6.216154,11.407,11.407
106172,23,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,5.401328,0.000127,0.000029,5.401328,213,23,5.401328,5.401328,11.407,11.407
106173,24,2011-08-01 23:00:00,28.094995,952.008506,-72.970251,-73.622512,0.0,0.543757,1.701971,0.004728,...,4.590689,0.000026,0.000037,4.590689,213,23,4.590689,4.590689,11.407,11.407


In [115]:
final_df.to_csv("all_data_lake_modeling_in_time.csv", index=False)